In [1]:
import pandas as pd
import numpy as np

import sys
import csv

In [2]:
points = pd.read_csv('../data/charting-m-points-from-2017.csv', quoting=csv.QUOTE_NONE, encoding = 'ISO-8859-1')
matches = pd.read_csv('../data/charting-m-matches.csv', quoting=csv.QUOTE_NONE, encoding = 'ISO-8859-1')

C:\Users\daniel.georgiev\AppData\Local\Temp\ipykernel_19628\15952938.py:1: DtypeWarning: Columns (8,9,29) have mixed types. Specify dtype option on import or set low_memory=False.
  points = pd.read_csv('../data/charting-m-points-from-2017.csv', quoting=csv.QUOTE_NONE, encoding = 'ISO-8859-1')


Merge the two datasets together by match_id

In [3]:
joinedData = pd.merge(
    matches,
    points,
    how="inner",
    on='match_id',
)

In [4]:
joinedData['rallyCount'] = pd.to_numeric(joinedData['rallyCount'], errors='coerce', downcast='integer').fillna(0).astype(int)
df_obj = joinedData.select_dtypes(['object'])
joinedData[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

Select relevant columns from dataframe


In [5]:
joinedData.columns = joinedData.columns.to_series().apply(lambda x: x.strip())
display(joinedData)
joinedData = joinedData.loc[:, ['match_id', 'Player 1', 'Player 2', 'Pl 1 hand', 'Pl 2 hand', 'Surface', 'Set1','Set2', 'Gm1', 'Gm2', 'Pts', 'Serving', '1st', '2nd', 'isAce', 'isUnret', 'isSvrWinner', 'rallyCount']]

,match_id,Player 1,Player 2,Pl 1 hand,Pl 2 hand,Gender,Date,Tournament,Round,Time,...,2ndIn,isAce,isUnret,isRallyWinner,isForced,isUnforced,isDouble,PtWinner,isSvrWinner,rallyCount
0,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,Marin Cilic,Pablo Carreno Busta,R,R,M,20221123,Davis Cup Finals,QF,18:35,...,0.0,False,False,False,False,False,True,2,0,0
1,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,Marin Cilic,Pablo Carreno Busta,R,R,M,20221123,Davis Cup Finals,QF,18:35,...,NaN,False,True,False,False,False,False,1,1,1
2,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,Marin Cilic,Pablo Carreno Busta,R,R,M,20221123,Davis Cup Finals,QF,18:35,...,1.0,False,False,False,True,False,False,2,0,2
3,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,Marin Cilic,Pablo Carreno Busta,R,R,M,20221123,Davis Cup Finals,QF,18:35,...,NaN,False,False,False,True,False,False,1,1,1
4,20221123-M-Davis_Cup_Finals-QF-Marin_Cilic-Pab...,Marin Cilic,Pablo Carreno Busta,R,R,M,20221123,Davis Cup Finals,QF,18:35,...,NaN,False,True,False,False,False,False,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295670,20170102-M-Doha-R32-Vasek_Pospisil-Fernando_Ve...,Vasek Pospisil,Fernando Verdasco,R,L,M,20170102,Doha,R32,22:25,...,NaN,False,False,True,False,False,False,1,0,6
295671,20170102-M-Doha-R32-Vasek_Pospisil-Fernando_Ve...,Vasek Pospisil,Fernando Verdasco,R,L,M,20170102,Doha,R32,22:25,...,1.0,False,False,False,False,True,False,2,1,3
295672,20170102-M-Doha-R32-Vasek_Pospisil-Fernando_Ve...,Vasek Pospisil,Fernando Verdasco,R,L,M,20170102,Doha,R32,22:25,...,0.0,False,False,False,False,False,True,1,0,0
295673,20170102-M-Doha-R32-Vasek_Pospisil-Fernando_Ve...,Vasek Pospisil,Fernando Verdasco,R,L,M,20170102,Doha,R32,22:25,...,NaN,False,False,False,True,False,False,2,1,3


In [6]:
joinedData['isAce'] = joinedData['isAce'] | joinedData['isUnret']
joinedData = joinedData.drop(columns=['isUnret', 'isAce'])

In [7]:
tennis_players = ['Daniil Medvedev', 'Alexander Zverev', 'Roger Federer', 'Dominic Thiem', 'Nick Kyrgios', 'Novak Djokovic', 'Rafael Nadal', 'Stefanos Tsitsipas', 'Casper Ruud', 'Grigor Dimitrov']
tennis_players_initials = [''.join([x[0].upper() for x in fullname.split(' ')]) for fullname in tennis_players]
joinedData = joinedData.loc[((joinedData['Player 1'].isin(tennis_players)) | (joinedData['Player 2'].isin(tennis_players)))]
joinedData = joinedData.loc[joinedData['Serving'].isin(tennis_players_initials)]

In [8]:
joinedData[['Pts1', 'Pts2']] = joinedData['Pts'].str.split('-', expand=True)
joinedData = joinedData.drop('Pts', axis='columns')

Separate the columns into new columns that are specific for Server and Returner.

In [9]:
server_names = []
returner_names = []
pointsServer = []
pointsReturner = []
gamesServer = []
gamesReturner = []
setsServer = []
setsReturner = []
handReturner = []
for index, row in joinedData.iterrows():
    if ''.join([x[0].upper() for x in row['Player 1'].split(' ')]) == row['Serving']:
      server_names.append(row['Player 1'])
      returner_names.append(row['Player 2'])
      pointsServer.append(row['Pts1'])
      pointsReturner.append(row['Pts2'])
      gamesServer.append(row['Gm1'])
      gamesReturner.append(row['Gm2'])
      setsServer.append(row['Set1'])
      setsReturner.append(row['Set2'])
      handReturner.append(row['Pl 2 hand'])
    elif ''.join([x[0].upper() for x in row['Player 2'].split(' ')]) == row['Serving']:
      server_names.append(row['Player 2'])
      returner_names.append(row['Player 1'])
      pointsServer.append(row['Pts2'])
      pointsReturner.append(row['Pts1'])
      gamesServer.append(row['Gm2'])
      gamesReturner.append(row['Gm1'])
      setsServer.append(row['Set2'])
      setsReturner.append(row['Set1'])
      handReturner.append(row['Pl 1 hand'])

joinedData['FullNameServer'] = server_names
joinedData['FullNameReturner'] = returner_names
joinedData['PointsServer'] = pointsServer
joinedData['PointsReturner'] = pointsReturner
joinedData['GamesServer'] = gamesServer
joinedData['GamesReturner'] = gamesReturner
joinedData['SetsServer'] = setsServer
joinedData['SetsReturner'] = setsReturner
joinedData['HandReturner'] = handReturner

joinedData = joinedData.drop(columns=['Player 1', 'Player 2', 'Pts1', 'Pts2', 'Gm1', 'Gm2', 'Set1', 'Set2', 'Pl 1 hand', 'Pl 2 hand', 'Serving', 'FullNameReturner'])

Combine the serves into one column with a list of 1 or 2 serves so that it can be exploded into several rows later

In [10]:
joinedData['Serves'] = joinedData.apply(lambda x: list(['f' + str(x['1st']),
                                                        's' + str(x['2nd'])]), axis=1)   

Explode list of first and second serve into separate rows

In [11]:
joinedData = joinedData.explode('Serves', ignore_index=True)

Add new column indicating whether the serve was first or second

In [12]:
joinedData['IsFirstServe'] = joinedData['Serves'].str[0] == 'f'

Remove 'f' and 's' (first and second serve) that were indicators before the explosion

In [13]:
joinedData['Serves'] = joinedData['Serves'].str[1:]
joinedData = joinedData[(joinedData['Serves'] != 'nan') & (joinedData['Serves'] != '')]
joinedData = joinedData.drop(columns=['1st', '2nd'])

Remove lets

In [14]:
joinedData['Serves'] = joinedData['Serves'].map(lambda x: x.lstrip('c'))

In [15]:
joinedData['FullNameServer'].value_counts()

FullNameServer
Roger Federer         12724
Novak Djokovic        12549
Daniil Medvedev       12427
Stefanos Tsitsipas    10837
Rafael Nadal          10717
Dominic Thiem         10051
Alexander Zverev       8100
Nick Kyrgios           7470
Casper Ruud            5300
Grigor Dimitrov        5101
Name: count, dtype: int64

Extract direction of serve

In [16]:
def extract_serve_direction(row):
  serves_directions_dict = {
      '4': 'Out wide',
      '5': 'Body',
      '6': 'Down the T'
  }

  if len(row['Serves']) > 1 and row['Serves'][1] in ['x', 'd', 'w', 'n', 'g', '!', 'e']:
      return "Fault"
  else:
      return serves_directions_dict.get(row['Serves'][0], 'Body')

joinedData['Direction'] = joinedData.apply(extract_serve_direction, axis=1)
joinedData = joinedData.drop(columns='Serves')

### Mark Faults as lost points with 0 rally length because otherwise the data has a vision on the future on the second serve.

In [17]:
joinedData.loc[joinedData['Direction'] == 'Fault', 'rallyCount'] = 0
joinedData.loc[joinedData['Direction'] == 'Fault', 'isSvrWinner'] = 0

### Fill arrays with previous serves information

In [18]:
def look_previous_serves(group):
    window = 5
    result = pd.DataFrame({
        'previous_directions': np.lib.stride_tricks.sliding_window_view(group['Direction'], window).tolist(),
        'previous_points_server_wins': np.lib.stride_tricks.sliding_window_view(group['isSvrWinner'], window).tolist(),
        'previous_rallies_lengths': np.lib.stride_tricks.sliding_window_view(group['rallyCount'], window).tolist(),
        'previous_serves_is_first': np.lib.stride_tricks.sliding_window_view(group['IsFirstServe'], window).tolist()
    })

    result = result[:-1]
    result.index = group.index[5:]
    
    group = group.join(result)
    
    return group

newDf = joinedData.groupby(by=['match_id', 'FullNameServer']).apply(look_previous_serves)

### Remove Faults

In [19]:
# joinedData = joinedData[joinedData['Direction'] != 'Fault']

### Remove columns that were used only for previous serves

In [20]:
newDf = newDf.drop(columns=['isSvrWinner', 'rallyCount', 'match_id'])

In [21]:
joinedData.to_csv('../data/wrangled-data.csv')
newDf.to_csv('../data/wrangled-data-with-previous-serves.csv', index=False)